---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    readonly/train.csv - the training set (all tickets issued 2004-2011)
    readonly/test.csv - the test set (all tickets issued 2012-2016)
    readonly/addresses.csv & readonly/latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `readonly/train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `readonly/test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32
       
### Hints

* Make sure your code is working before submitting it to the autograder.

* Print out your result to see whether there is anything weird (e.g., all probabilities are the same).

* Generally the total runtime should be less than 10 mins. You should NOT use Neural Network related classifiers (e.g., MLPClassifier) in this question. 

* Try to avoid global variables. If you have other functions besides blight_model, you should move those functions inside the scope of blight_model.

* Refer to the pinned threads in Week 4's discussion forum when there is something you could not figure it out.

In [23]:
import pandas as pd
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, auc
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
#import matplotlib.pyplot as plt
def blight_model():
    dico={'knn':KNeighborsClassifier,'lin':LinearRegression,'ridge':Ridge,
          'lasso':Lasso,'poly':PolynomialFeatures,'log':LogisticRegression,
          'svc':SVC,'linsvc':LinearSVC,'naiveb':GaussianNB,'forest':RandomForestClassifier,
          'gtree':GradientBoostingClassifier,'neural':MLPClassifier}
    
    dftrain=pd.read_csv('train.csv',encoding='latin1')
    dftest=pd.read_csv('test.csv')
    dftest=dftest.set_index('ticket_id')
    dftrain=dftrain.set_index('ticket_id')
    dftrain=dftrain.drop(['agency_name','inspector_name', 'violator_name','violation_street_number', 'violation_street_name','mailing_address_str_number',
       'mailing_address_str_name','non_us_str_code','violation_description','admin_fee', 'state_fee', 'late_fee','payment_amount', 'balance_due',
       'payment_date', 'payment_status', 'city','state','disposition','country','collection_status','violation_zip_code','violation_code', 'compliance_detail','ticket_issued_date', 'hearing_date','zip_code'],axis=1)
    dftest=dftest.drop(['agency_name','inspector_name', 'violator_name','violation_street_number', 'violation_street_name','mailing_address_str_number',
       'mailing_address_str_name','non_us_str_code','country','disposition','violation_description','admin_fee','violation_code', 'state_fee', 'late_fee', 'city','state','violation_zip_code','ticket_issued_date', 'hearing_date','zip_code'],axis=1)
    dftrain2=pd.get_dummies(dftrain,columns=['grafitti_status'])
    dftrain2=dftrain2.dropna()
    dftest2=pd.get_dummies(dftest,columns=['grafitti_status'])
    X=dftrain2[[i for i in list(dftrain2.columns) if i != 'compliance']]
    y=dftrain2['compliance']
    X2=dftest2
    clf=dico['forest']()
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)    
    clf.fit(X_train,y_train)
    a=clf.score(X_test, y_test)
    #clf.score(X_test, y_test)
    #cv_scores = cross_val_score(clf, X, y)
    y_score_clf = clf.predict_proba(X_test)
    y2_score_clf = clf.predict_proba(X2)
    X2=X2.reset_index()
    y_score_df=pd.DataFrame(data=y_score_clf)
    y2_score_df = pd.DataFrame(data=y2_score_clf)
    X2['compliance']=y2_score_df[1]
    X2=X2.set_index('ticket_id')
    X2=X2.drop(['fine_amount', 'discount_amount', 'clean_up_cost', 'judgment_amount','grafitti_status_GRAFFITI TICKET'],axis=1)

#    fpr_clf, tpr_clf, _ = roc_curve(y_test, y_score_df[1])
#    roc_auc_clf = auc(fpr_clf, tpr_clf)
    
#    plt.figure()
#    plt.xlim([-0.01, 1.00])
#    plt.ylim([-0.01, 1.01])
#    plt.plot(fpr_clf, tpr_clf, lw=3, label='clf ROC curve (area = {:0.2f})'.format(roc_auc_clf))
#    plt.xlabel('False Positive Rate', fontsize=16)
#    plt.ylabel('True Positive Rate', fontsize=16)
#    plt.title('ROC curve (1-of-10 digits classifier)', fontsize=16)
#    plt.legend(loc='lower right', fontsize=13)
#    plt.plot([0, 1], [0, 1], color='navy', lw=3, linestyle='--')
#    plt.axes().set_aspect('equal')
#    plt.show()

    return X2

blight_model()# Your answer here

/opt/conda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


,compliance
ticket_id,
284932,0.060507
285362,0.024216
285361,0.071129
285338,0.060507
285346,0.071129
285345,0.060507
285347,0.055705
285342,0.362785
285530,0.024216


In [ ]:
blight_model()